In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

# Change Directory to file location

In [2]:
os.chdir(r'C:\Users\kjsdr\Documents\GitHub\foster-app\sample-data\Sadhanand')

# Read file content into Pandas DataFrame

In [3]:
dataset=pd.read_excel('mavoix_ml_sample_dataset.xlsx')

# Have a look at the Data

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 23 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        374 non-null object
Current 

# Looks like there are some null values for some of the columns.Need to Check whether all null values have been captured(those marked as N/A in excel file as well).check row for application ID ML16

In [8]:
id_filt=(dataset['Application_ID']=='ML0016')
dataset.loc[id_filt,['Performance_UG','Performance_12']]

,Performance_UG,Performance_12
15,8.00/10,NaN


# check empty space capture.check row for ML0025

In [9]:
id_filt=(dataset['Application_ID']=='ML0025')
dataset.loc[id_filt,['Stream']]

,Stream
24,NaN


# check percentage of NaN values for each column

In [10]:
dataset.isna().mean().round(4) * 100

Application_ID                 0.00
Current City                   0.00
Python (out of 3)              0.00
R Programming (out of 3)       0.00
Deep Learning (out of 3)       0.00
PHP (out of 3)                 0.00
MySQL (out of 3)               0.00
HTML (out of 3)                0.00
CSS (out of 3)                 0.00
JavaScript (out of 3)          0.00
AJAX (out of 3)                0.00
Bootstrap (out of 3)           0.00
MongoDB (out of 3)             0.00
Node.js (out of 3)             0.00
ReactJS (out of 3)             0.00
Other skills                   3.57
Degree                         3.06
Stream                         4.59
Current Year Of Graduation     0.00
Performance_PG                73.72
Performance_UG                13.78
Performance_12                41.58
Performance_10                45.15
dtype: float64

# even though there is a high percentage of missing values in Performance_ metrics,we are not going to remove those columns,since we deem it important for judging a candidate.

# going to impute missing values with model based imputation since it aint appropriate to take the mean/median here since large part of the columns have missing values.instead,imputation is done by taking those rows that have values for columns as training set,and predicting the values for missing data.

In [11]:
# Performance UG
from sklearn.impute import KNNImputer

ImportError: cannot import name 'KNNImputer' from 'sklearn.impute' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\impute\__init__.py)